In [2]:
if __name__ == '__main__':
    import torch
    import numpy as np
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
    from torch.utils.data import TensorDataset
    import networkx as nx
    from karateclub import DeepWalk, NetMF, GLEE, Node2Vec, Diff2Vec
    from torchmetrics import MeanAbsolutePercentageError
    import matplotlib.pyplot as plt
    from argparse import ArgumentParser
    import os
    #from torchsummary import summary
    from dataset import *
    from models import *
    
    parser = ArgumentParser()
    parser.add_argument('--accelerator', type=str, default="gpu")
    parser.add_argument('--lr', type=float, default=0.01)
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--max_epochs', type=int, default=75)
    parser.add_argument('--device_num', type=int, default=0)
    parser.add_argument('--model', type=str, default='dense')
    parser.add_argument('--embedding', type=str, default='all')

    args = parser.parse_args()
    
    accelerator = args.accelerator
    lr = args.lr
    batch_size = args.batch_size
    max_epochs = args.max_epochs
    device_num = args.device_num
    model = args.model
    embedding = args.embedding
    
    
    
    adj_mtx = nx.read_gpickle("git/DCRNN/data/sensor_graph/adj_mx.pkl")[2]
    adj_mtx = nx.from_numpy_matrix(np.matrix(adj_mtx))
    deepwalk = DeepWalk()
    glee = GLEE()
    netmf = NetMF()
    node2vec = Node2Vec()
    diff2vec = Diff2Vec()
    
    embeddings = {'deepwalk':deepwalk,'glee':glee,'netmf':netmf,'diff2vec':diff2vec,'none':None}
    if embedding == 'all':
        embedding_list = [deepwalk, glee, netmf, diff2vec, None]
    else:
        embedding_list = [embeddings[embedding]]
        
        
    LSTMtrainset = LSTMDataset(np.load('data/METR-LA/train.npz'), adj_mtx = adj_mtx, embedding = deepwalk, scaling = True)
    LSTMtestset = LSTMDataset(np.load('data/METR-LA/test.npz'), adj_mtx = adj_mtx, embedding = deepwalk,scaling = True)
    LSTMvalset = LSTMDataset(np.load('data/METR-LA/val.npz'), adj_mtx = adj_mtx, embedding = deepwalk,scaling = True)
    
    use_cuda = True
    device = torch.device("cuda:0" if use_cuda else "cpu")
    for embedding in embedding_list:
        for scaling in (True, False):
            print(f"Embedding: {embedding} | Scaling: {scaling}")
            LSTMtrainset.set_embedding(embedding)
            LSTMtestset.set_embedding(embedding)
            LSTMvalset.set_embedding(embedding)
            LSTMtrainset.set_scale(scaling)
            LSTMtestset.set_scale(scaling)
            LSTMvalset.set_scale(scaling)

            if embedding:
                if embedding == glee:
                    features = 2+embedding.dimensions+1
                else:
                    features = 2+embedding.dimensions
            else:
                features = 2
            densemodel = DenseModel((12,features,207), 2*207, batch_size=batch_size).to(device)
            optimizer =  torch.optim.Adam(densemodel.parameters(), 0.01, amsgrad = True, weight_decay=0)  
            #optimizer = torch.optim.SGD(lstmmodel.parameters(), lr=0.1, momentum=0.1)

            for i in range(max_epochs):
                densemodel, trainpred, traintruth, scores = train(model=densemodel, optimizer=optimizer, trainset=LSTMtrainset, batch_size=batch_size, current_epoch=i, lstm=False)
                pred, truth, scores = evaluate(model=densemodel, optimizer=optimizer, batch_size=batch_size, testset=LSTMvalset,current_epoch=i, lstm=False)
            print("Test:")
            pred, truth, scores = evaluate(model=densemodel, optimizer=optimizer, batch_size=batch_size, testset=LSTMtestset,current_epoch=i, lstm=False)


